In [11]:
# Add to environment variables PYTHONPATH = C:\Users\JFC_WIN\Documents\OpenSimula\src
import OpenSimula as osm
import pandas as pd

bestest_building_600 = {
    "name": "Bestest Building",
    "time_step": 3600,
    "n_time_steps": 8760,
    "initial_time": "01/01/2001 00:00:00",
    "components": [
        # MET_FILE
        {
            "type": "File_met",
            "name": "sevilla",
            "file_name": "met_files/sevilla.met"
        },
        # CONSTRUCTION
        {
            "type": "Material",
            "name": "Enlucido",
            "conductivity": 0.16,
            "density": 950,
            "specific_heat": 840
        },
        {
            "type": "Material",
            "name": "Aislamiento",
            "conductivity": 0.04,
            "density": 12,
            "specific_heat": 840
        },
        {
            "type": "Material",
            "name": "Madera pared",
            "conductivity": 0.14,
            "density": 530,
            "specific_heat": 900
        },
        {
            "type": "Material",
            "name": "Aislamiento suelo",
            "conductivity": 0.04,
            "density": .1,
            "specific_heat": .1,
        },
        {
            "type": "Material",
            "name": "Madera suelo",
            "conductivity": 0.14,
            "density": 650,
            "specific_heat": 1200,
        },
        {
            "type": "Material",
            "name": "Lecho_rocas",
            "conductivity": 0.14,
            "density": 650,
            "specific_heat": 1200,
        },
        {
            "type": "Construction",
            "name": "Pared 600",
            "solar_absortivity": [0.6, 0.6],
            "materials": ["Enlucido", "Aislamiento", "Madera pared"],
            "thicknesses": [0.12, 0.066, 0.009],
        },
        {
            "type": "Construction",
            "name": "Suelo 600",
            "solar_absortivity": [0.6, 0.6],
            "materials": ["Aislamiento suelo", "Madera suelo"],
            "thicknesses": [1.003, 0.025],
        },
        {
            "type": "Construction",
            "name": "Techo 600",
            "solar_absortivity": [0.6, 0.6],
            "materials": ["Enlucido", "Aislamiento", "Madera pared"],
            "thicknesses": [0.010, 0.01118, 0.019],
        },
        {
            "type": "Window",
            "name": "Vidrio doble",
            "solar_absortivity": [0.13, 0.13],
            "solar_transmisivity": [0.66, 0.66],
            "R_glazing": 0.921,
            "R_frame": 0.921,
            "frame_fraction": 0
        },
        # BUILDING
        {
            "type": "Building",
            "name": "Bestest 600",
            "file_met": "sevilla"
        },
        {
            "type": "Space_type",
            "name": "bestest space",
            "aux_variables": [],
            "people_density": "0",
            "light_density": "0",
            "other_gains_density": "4.1667",
            "other_gains_radiant_fraction": 0.6,
            "infiltration": "0.5"
        },
        {
            "type": "Space",
            "name": "space_1",
            "building": "Bestest 600",
            "space_type": "bestest space",
            "floor_area": 48,
            "volume": 48*2.7,
        },
        # Surfaces
        {
            "type": "Exterior_surface",
            "name": "north_wall",
            "virtual": False,
            "construction": "Pared 600",
            "space": "space_1",
            "area": 8*2.7,
            "azimuth": 180,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "east_wall",
            "construction": "Pared 600",
            "space": "space_1",
            "area": 6*2.7,
            "azimuth": 90,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "south_wall",
            "construction": "Pared 600",
            "space": "space_1",
            "area": 8*2.7,
            "azimuth": 0,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "west_wall",
            "construction": "Pared 600",
            "space": "space_1",
            "area": 6*2.7,
            "azimuth": -90,
            "altitude": 0
        },
        {
            "type": "Exterior_surface",
            "name": "roof",
            "construction": "Techo 600",
            "space": "space_1",
            "area": 8*6,
            "azimuth": 0,
            "altitude": 90
        },
        {
            "type": "Underground_surface",
            "name": "floor",
            "construction": "Suelo 600",
            "space": "space_1",
            "area": 8*6,
            "azimuth": 0,
            "altitude": -90
        },
    ],
}

In [12]:
sim = osm.Simulation()
pro = osm.Project("pro", sim)
pro.read_dict(bestest_building_600)
pro.simulate()

Reading project data from dictonary
Reading completed.
Checking project: Bestest Building
ok
Simulating Bestest Building: 10% 20% 30% 40% 50% 60% 70% 80% 90% 100%  End


In [13]:
pro.component("roof").K


-282.49283959547734

In [14]:
df_met = pro.component("sevilla").variable_dataframe()
df_space = pro.component("space_1").variable_dataframe()
df_wall = pro.component("roof").variable_dataframe()

df_graph = pd.DataFrame()
df_graph["date"] = df_met["date"]
df_graph["T_ext"] = df_met["temperature [°C]"]
df_graph["T_z"] = df_space["temperature [°C]"]
df_graph["T_s0"] = df_wall["T_s0 [°C]"]
df_graph["T_s1"] = df_wall["T_s1 [°C]"]

df_wall.head(24)

,date,T_s0 [°C],T_s1 [°C],T_rm [°C],E_dir0 [W/m²],E_dif0 [W/m²],q_cd0 [W/m²],q_cd1 [W/m²],p_0 [W/m²],p_1 [W/m²]
0,2001-01-01 00:00:00,5.627997,16.506310,-8.637367,0.000000,0.000000,53.277041,-10.007786,48.944031,46.358247
1,2001-01-01 01:00:00,4.291117,13.859432,-8.606039,0.000000,0.000000,23.863602,-14.454308,18.439337,33.596135
2,2001-01-01 02:00:00,12.191310,14.199450,3.673309,0.000000,0.000000,-11.886898,-8.233798,14.605351,28.021627
3,2001-01-01 03:00:00,12.337021,14.371567,3.173309,0.000000,0.000000,3.890713,-4.647206,30.695592,32.051124
4,2001-01-01 04:00:00,11.901834,14.105166,2.667971,0.000000,0.000000,5.604818,-3.922392,31.063780,32.439371
5,2001-01-01 05:00:00,11.334097,13.694288,2.073309,0.000000,0.000000,6.262716,-3.920855,30.071235,31.749602
6,2001-01-01 06:00:00,10.834077,13.253119,1.573309,0.000000,0.000000,6.262241,-4.015837,28.749161,30.729980
7,2001-01-01 07:00:00,10.337938,12.804198,1.073309,0.000000,0.000000,6.356577,-4.101702,27.550754,29.682148
8,2001-01-01 08:00:00,9.892342,12.361562,0.616014,0.000000,0.640580,6.258069,-4.141448,26.356854,28.621338
9,2001-01-01 09:00:00,10.438674,12.154170,0.964058,0.127754,14.722463,2.577386,-3.433496,25.267657,27.594072


In [15]:
import plotly.express as px

fig = px.line(df_graph,x='date',y=['T_ext','T_z','T_s0','T_s1'],height=400)
fig.show()